In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

import warnings
warnings.filterwarnings("ignore")

In [ ]:
mnist = input_data.read_data_sets('MNIST.data', one_hot=True)

batch_size = 100
n_batch = mnist.train.num_examples // batch_size
n_test_batch = mnist.train.num_examples // batch_size


def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    #x 有4维。[batch, 长, 宽, 深度]
    #W 过滤器4维。 [长, 宽, 输入深度, 输出深度]
    #tf.nn.conv2d同样传入4个参数 conv2d(x,w,strides=[4个参数], padding='SAME)  strides步长。只涉及中间两个参数，左右/上下走
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')

#池化层
def max_pool_2x2(x):
    #ksize=[1,x,y,1]
    return tf.nn.max_pool(x, ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

#定义输入输出
x = tf.placeholder(tf.float32,[None,784])
y = tf.placeholder(tf.float32,[None,10])

#转换下images的格式
x_image = tf.reshape(x, [-1,28,28,1])

#初始化第一个卷积的权值和偏置
W_conv1 = weight_variable([5,5,1,32])  #5*5的卷积核。输入深度为1(黑白的).输出深度为32.有32个这样的卷积核
b_conv1 = bias_variable([32])

#进行卷积计算，加上偏置值。然后    使用   relu激活函数
h_conv1 = tf.nn.relu(conv2d(x_image,W_conv1)+b_conv1)

#进行最大池化
h_pool1 = max_pool_2x2(h_conv1)

#定义第二层的权重和偏置
W_conv2 = weight_variable([5,5,32,64])  #5*5的卷积核。输入深度为1(黑白的).输出深度为32.有32个这样的卷积核
b_conv2 = bias_variable([64])

#进行卷积计算，加上偏置值。然后使用relu激活函数
h_conv2 = tf.nn.relu(conv2d(h_pool1,W_conv2)+b_conv2)

#进行最大池化
h_pool2 = max_pool_2x2(h_conv2)

#28*28的原始图片。第一次卷积后还是28*28；第一次池化后14*14.  第二次卷积后14*14；第二次池化后7*7  共64个7*7

#全连接层

#初始化第一个全连接层的权重
W_fc1 = weight_variable([7*7*64, 1024])
b_fc1 = bias_variable([1024])

#将池化层2层压缩为全连接层
h_pool2_float = tf.reshape(h_pool2, [-1,7*7*64])

#求第一个全连接层的输出
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_float, W_fc1) + b_fc1)

#加入dropout层
keep_prob = tf.placeholder(tf.float32)
h_fc1_prob = tf.nn.dropout(h_fc1, keep_prob)

#初始化第二个全连接层的权重
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

#计算输出
prediction = tf.nn.softmax(tf.matmul(h_fc1_prob,W_fc2)+b_fc2)

#交叉熵
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=prediction, labels=y))

#使用AdamOptimizer进行优化
train_step = tf.train.AdamOptimizer(0.0001).minimize(cross_entropy)

correct_predition = tf.equal(tf.argmax(y,1), tf.argmax(prediction,1))
accuracy = tf.reduce_mean(tf.cast(correct_predition, tf.float32))

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(21):
        for batch in range(n_batch):
            batch_xs,batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step, feed_dict={x:batch_xs, y:batch_ys, keep_prob:0.8})
        acc = 0
        for batch in range(n_test_batch):
            batch_test_xs,batch_test_ys = mnist.test.next_batch(batch_size)
            acc += sess.run(accuracy, feed_dict={x: batch_test_xs, y: batch_test_ys,keep_prob:1})
        print("Iter " + str(epoch) + ",Testing Accuracy " + str(acc/(batch+1)))

当GPU显存不够大时。在train数据集分批次后。test也要分批次计算。

n_test_batch = mnist.train.num_examples // batch_size
        acc = 0
        for batch in range(n_test_batch):
            batch_test_xs,batch_test_ys = mnist.test.next_batch(batch_size)
            acc += sess.run(accuracy, feed_dict={x: batch_test_xs, y: batch_test_ys,keep_prob:1})